In [31]:
!python -V

Python 3.10.9


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle

In [3]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='/home/danial/Desktop/mymlops/MLOPS/experiment-tracking/mlruns/1', creation_time=1684962248383, experiment_id='1', last_update_time=1684962248383, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(file_name):
    df = pd.read_parquet(file_name)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60)
    df = df[((df.duration >= 1) & (df.duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] =  df[categorical].astype(str)

    return df 

In [5]:
parent_directory = '/home/danial/Desktop/mymlops/MLOPS/data/' 

In [6]:
df_train = read_dataframe(parent_directory + 'green_tripdata_2021-01.parquet')
df_val = read_dataframe(parent_directory + 'green_tripdata_2021-02.parquet')
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
numerical = ['trip_distance']
categorical = ['PU_DO']

In [9]:
df_train[categorical + numerical]

,PU_DO,trip_distance
0,43_151,1.01
1,166_239,2.53
2,41_42,1.12
3,168_75,1.99
7,75_75,0.45
...,...,...
76513,81_90,17.63
76514,35_213,18.36
76515,74_69,2.50
76516,168_215,14.48


In [10]:
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)



val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)


In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [52]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)



7.758715202423387

In [53]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
with mlflow.start_run():

    mlflow.set_tag("developer", 'danial')

    mlflow.log_param('train-data-path', parent_directory + 'green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', parent_directory + 'green_tripdata_2021-02.parquet')

    alpha = 1
    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

In [187]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7037351333448765

In [203]:
len(df_train['PU_DO'].unique())

13220

In [13]:
import xgboost as xgb

In [14]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[10:42:16] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.59374                          
[1]	validation-rmse:18.15210                          
[2]	validation-rmse:16.85377                          
[3]	validation-rmse:15.68707                          
[4]	validation-rmse:14.64046                          
[5]	validation-rmse:13.70269                          
[6]	validation-rmse:12.86452                          
[7]	validation-rmse:12.11872                          
[8]	validation-rmse:11.45577                          
[9]	validation-rmse:10.86603                          
[10]	validation-rmse:10.34435                         
[11]	validation-rmse:9.88514                          
[12]	validation-rmse:9.47878                          
[13]	validation-rmse:9.12218                          
[14]	validation-rmse:8.80847                          
[15]	validation-rmse:8.53415                          
[

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")